In [ ]:
import pickle as p
import numpy as np
from plotly import graph_objects as go
import os
from sklearn.mixture import GaussianMixture
from tqdm.notebook import tqdm  # last tqdm update
from scipy.stats import shapiro
from statsmodels.sandbox.stats.multicomp import multipletests 

In [ ]:
first_ten = []
for file in os.listdir('./Adam'):
    with open('./Adam/' + file, 'rb') as inp:
        try:
            first_ten.append(p.load(inp)[0][:10])
        except:
            pass
first_ten = np.array(first_ten)
first_ten.shape

# Распределение градиентов

## Данные

Для первых ста итераций обучения были получены значения градиентов для весов между десятью нейронами на последнем слое и
Предпоследним скрытым слоем с 512 нейронами.

## Первый взгляд

Давайте посмотрим на все градиенты для каждого нейрона за все итерации. Такая апроксимация не слишком математична, так как для каждого веса распределение свое, однако такая интерперетация более наглядна. Полученная величина -- смесь нормальных распределений. Оценим эту смесь через `GaussianMixture` и построим гистограммы.

In [ ]:
from plotly.subplots import make_subplots



data = []
all_grad = []
mixt_est = []


for param in range(10):
    param_grad = first_ten[:, param, :].reshape(-1)
    all_grad.append(param_grad)



models = []

for it in range(10):
    data.append(go.Histogram({
        'x': all_grad[it],
        'opacity': 0.7,
        'histnorm': 'probability density',
        'name': 'param: {} all weights'.format(it+1)
    }))
    cur_model = GaussianMixture(5)
    cur_model.fit(all_grad[it].reshape(-1, 1))
    x = np.linspace(np.min(all_grad[it]), np.max(all_grad[it]), 200).reshape(-1, 1)
    y = cur_model.score_samples(x).reshape(-1)
    if it == 0:
        mixt_est.append(go.Scatter({
            'x': x.reshape(-1),
            'y': np.exp(y),
            'marker': {
                'color': 'red'
            },
        
            'name': 'Gaussian mixture estimator'
        }))
        continue
        
    mixt_est.append(go.Scatter({
        'x': x.reshape(-1),
        'y': np.exp(y),
        'marker': {
            'color': 'red'
        },
        
        'showlegend': False
    }))
    

fig = make_subplots(cols=4, rows=3)

for idx, trace in enumerate(data):
    fig.add_trace(mixt_est[idx], col=idx//3 + 1, row=idx%3+1)
    fig.add_trace(trace, col=idx//3 + 1, row=idx%3+1)
    
    
layout = go.Layout({
    'title': 'Distribution of gradients',
    'width': 1000,
    'height': 800,
    'template': 'plotly_white',
    #'legend_orientation': 'h',
    'legend': {
        'x': 0.8,
        'y': 0.4
    }
})
fig.update_layout(layout)
fig.show()

## Вывод

Как видим данные вполне неплохо подходят под данное распределение, а значит мы можем продолжить нашу проверку на нормальность градиентов каждого веса.

## Критерий согласия Шапиро-Уилка

Для каждого веса для каждого нейрона проверим гипотезу $H_0: \text{"Градиенты на данном весе распределены нормально"} $ против альтернативы $H_1: \text{"Иначе"} $. Для этого воспользуемся критерием согласия Шапиро-Уилка.

In [ ]:
alpha = 0.05

p_values = []
for param in tqdm(range(10)):
    p_values_param = []
    for weight in range(512):
        p_value = shapiro(first_ten[:, param, weight])[1]
        p_values_param.append(p_value)
    p_values.append(p_values_param)

p_values = np.array(p_values)

Итак, число не отвергнутых гипотез и отвергнутых гипотез можем видеть в выводе следующе ячейки.

In [ ]:
accepted = p_values[p_values > alpha]
rejected = p_values[p_values <= alpha]
len(accepted), len(rejected)

Так же посмотрим на каждый нейрон и веса, принадлежащие конкретному нейрону.

In [ ]:
for idx, param_p in enumerate(p_values):
    accepted = len(param_p[param_p > alpha])
    rejected = len(param_p[param_p <= alpha])
    print('For parameter {} Shapiro-Wilk test results are:\n accepted {}; rejected {}; percentage accepted {:.2f}'.format(
    idx+1, accepted, rejected, accepted/len(param_p)*100))

## Множественная проверка гипотез

Воспользуемся поправками Холма и Бенджамини-Хохберга. Посмотрим какая из них даст более мощный результат и воспользуемся такой.

Будем проверять гипотезу $H_0: \text{"В целом градиенты распределены нормально"} $ против $ H_1: \text{"Иначе"} $

### Метод Холма

In [ ]:
reject, p_corrected, a1, a2 = multipletests(p_values.reshape(-1), 
                                            alpha = 0.05, 
                                            method = 'holm')

In [ ]:
reject.sum()

### Метод Бенджамини-Хохберга

In [ ]:
reject, p_corrected, a1, a2 = multipletests(p_values.reshape(-1), 
                                            alpha = 0.05, 
                                            method = 'fdr_bh')

In [ ]:
reject.sum()

### Вывод 
С помощью метода Бенджамини-Хохберга удалось отвергнуть больше гипотез, а значит он является более мощным в данном случае.